In [25]:
import functools
import itertools

import numpy as np

from pycobyla.cobyla import Cobyla


def gaussian(x, mu=None, sig=None, A=1):
    n = len(x)
    mu =  np.zeros(n) if mu is None else mu
    sig = np.ones(n) if sig is None else sig

    zz = ((((x - mu) / sig) ** 2) / 2).sum()
    return A * np.exp(-zz)


def neg_gaussian(x, mu=None, sig=None, A=1):
    return -gaussian(x, mu=mu, sig=sig, A=A)

In [31]:
def optimization_out_of_the_boundary_circle(radius=2):
    sig = np.random.choice((-1, 1))
    phi = np.random.uniform(low=0, high=2*np.pi*radius) / radius
    x_circle = radius * np.cos(phi)
    y_circle = radius * np.sin(phi)
    
    mu = np.array((x_circle, y_circle))
    
    G = functools.partial(neg_gaussian, mu=mu)
    c1 = lambda x: 1 - sum(x ** 2)

    C = (c1,)
    x = np.zeros(2)

    opt = Cobyla(x, G, C, rhobeg=0.5, rhoend=1e-12)
    opt.run()
    
    return mu, opt.x

In [39]:
%matplotlib widget
import matplotlib.pyplot as plt

n_targets = 200
data = tuple(optimization_out_of_the_boundary_circle() for _ in range(n_targets))
v_mu = np.array(tuple(mu for mu, opt in data))
v_opt = np.array(tuple(opt for mu, opt in data))

plt.close('all')
fig, ax = plt.subplots(figsize=(6, 6))
ax.axis('equal');

boundary = plt.Circle((0, 0), 1, color='y', fill=True)
targets = plt.Circle((0, 0), 2, color='b', fill=False)
ax.add_patch(boundary)
ax.add_patch(targets)

ax.plot(v_mu[:, 0], v_mu[:, 1], 'o')
ax.plot(v_opt[:, 0], v_opt[:, 1], 'x')
for mu, opt in zip(v_mu, v_opt):
    mu_x, mu_y = mu
    opt_x, opt_y = opt
    xx = np.array((mu_x, opt_x))
    yy = np.array((mu_y, opt_y))
    ax.plot(xx, yy, linestyle='-.')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [36]:
def optimization_out_of_the_boundary_square(side=1):
    radius = 2
    phi = np.random.uniform(low=0, high=2*np.pi*radius) / radius
    x_circle = radius * np.cos(phi)
    y_circle = radius * np.sin(phi)
    
    mu = np.array((x_circle, y_circle))
    
    G = functools.partial(neg_gaussian, mu=mu)
    c1 = lambda x: side/2 - x[0]
    c2 = lambda x: side/2 + x[0]
    c3 = lambda x: side/2 - x[1]
    c4 = lambda x: side/2 + x[1]

    C = (c1, c2, c3, c4)
    x = np.zeros(2)

    opt = Cobyla(x, G, C, rhoend=1e-12)
    opt.run()
    
    return mu, opt.x

In [38]:
%matplotlib widget
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

side = 2
n_targets = 200
data = tuple(optimization_out_of_the_boundary_square(side=side) for _ in range(n_targets))
v_mu = np.array(tuple(mu for mu, opt in data))
v_opt = np.array(tuple(opt for mu, opt in data))

plt.close('all')
fig, ax = plt.subplots(figsize=(6, 6))
ax.axis('equal')

boundary = plt.Rectangle((-1, -1), side, side, color='y', fill=True)
targets = plt.Circle((0, 0), 2, color='b', fill=False)
ax.add_patch(boundary)
ax.add_patch(targets)

ax.plot(v_mu[:, 0], v_mu[:, 1], 'o')
ax.plot(v_opt[:, 0], v_opt[:, 1], 'x')
for mu, opt in zip(v_mu, v_opt):
    mu_x, mu_y = mu
    opt_x, opt_y = opt
    xx = np.array((mu_x, opt_x))
    yy = np.array((mu_y, opt_y))
    ax.plot(xx, yy, linestyle='-.')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …